In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

ImportError: generic_type: type "_InterpolationType" is already registered!

In [14]:
# Define a neural network with a setup of 3 layers, each with 50 nodes. Use the Tanh activation function. Input time t, and output position y.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(1, 50)
        self.hidden_layer2 = nn.Linear(50, 50)
        self.hidden_layer3 = nn.Linear(50, 50)
        self.output_layer = nn.Linear(50, 1)

    def forward(self, t):
        outputs = torch.tanh(self.hidden_layer1(t))
        outputs = torch.tanh(self.hidden_layer2(outputs))
        outputs = torch.tanh(self.hidden_layer3(outputs))
        outputs = self.output_layer(outputs)
        return outputs

In [15]:
# Define the loss function, which includes physical loss and initial condition loss. The physical loss represents the adherence of motion to a certain physical law, while the initial condition loss represents the initial conditions that the motion of an object must satisfy.
# The weights for physical loss and initial condition loss can be adjusted.

def PINN_loss(t, net, h, t_end):
    g = 9.8
    y_pred = net(t)

    # calculate dy/dt
    dy_dt = torch.autograd.grad(y_pred, t, grad_outputs=torch.ones_like(y_pred),
                                create_graph=True)[0]

    # calculate d^2y/dt^2
    d2y_dt2 = torch.autograd.grad(dy_dt, t, grad_outputs=torch.ones_like(dy_dt),
                                  create_graph=True)[0]

    physics_loss = torch.mean((d2y_dt2 + g)**2) # pay attention to signs

    # initial loss
    y0 = net(torch.zeros(1, 1).requires_grad_(True))
    v0 = torch.autograd.grad(y_pred, t, grad_outputs=torch.ones_like(y_pred),
                             create_graph=True)[0][0]
    initial_loss = (y0 - h)**2 + v0**2

    # total loss
    total_loss = physics_loss + initial_loss

    return total_loss

In [16]:
# initial conditions

h = 100  # initial height
t_end = torch.tensor(6.0) # ending time

# generate training data
t_numpy = np.linspace(0, t_end.item(), 100).reshape(-1, 1)
y_numpy = h - 0.5 * 9.8 * t_numpy**2

# convert to PyTorch tensors
t = torch.tensor(t_numpy).requires_grad_(True)
y = torch.tensor(y_numpy)

In [17]:
net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [18]:
epochs = 20000
loss_history = []
for epoch in range(epochs):
    optimizer.zero_grad()

    t = torch.linspace(0, t_end.item(), 100).reshape(-1, 1).requires_grad_(True)
    total_loss = PINN_loss(t, net, h, t_end)

    total_loss.backward()
    optimizer.step()
    loss_history.append(total_loss.item())
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1}, Total Loss: {total_loss.item():.4f}')

Epoch 1000, Total Loss: 2580.1294
Epoch 2000, Total Loss: 405.8194
Epoch 3000, Total Loss: 21.2433


KeyboardInterrupt: 

In [1]:
# test model
t_test = torch.linspace(0, t_end.item(), 200).reshape(-1, 1).requires_grad_(True)
y_pred = net(t_test).detach().numpy()

# calculate analytical result
y_analytical = h - 0.5 * 9.8 * t_test.detach().numpy()**2

# visualization
plt.figure(figsize=(12, 8))
plt.scatter(t_numpy, y_numpy, label='Training Data', alpha=0.5)
plt.plot(t_test.detach().numpy(), y_pred, label='PINN Prediction', color='red')
plt.plot(t_test.detach().numpy(), y_analytical, label='Analytical Solution', color='green', linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Height (m)')
plt.legend()
plt.title(f'Free Fall Motion from {h}m: PINN vs Analytical Solution')
plt.grid(True)
plt.show()

NameError: name 'torch' is not defined